# Fetching air quality data from the FMI open data timeseries API

[API documentation](https://github.com/fmidev/smartmet-plugin-timeseries/blob/master/docs/Using-the-Timeseries-API.md),
[API examples](https://github.com/fmidev/smartmet-plugin-timeseries/blob/master/docs/Examples.md),
[JSON API example call](https://opendata.fmi.fi/timeseries?format=json&groupareas=0&producer=airquality_urban&area=Helsinki&param=time,fmisid,PM10_PT1H_avg,PM25_PT1H_avg,O3_PT1H_avg,CO_PT1H_avg,SO2_PT1H_avg,NO2_PT1H_avg,TRSC_PT1H_avg),
[CSV API call for the fmisid to name mapping](https://opendata.fmi.fi/timeseries?format=ascii&groupareas=0&separator=,&producer=airquality_urban&area=Finland&param=fmisid,name,latitude,longitude&starttime=2022-08-26T08:00:00%2B00:00&endtime=2022-08-26T08:00:00%2B00:00&tz=UTC).

In [1]:
import requests
import pendulum
import pandas as pd
import numpy as np

In [2]:
start_time = pendulum.yesterday('UTC')
start_time = pendulum.now('UTC').subtract(days=5)
end_time = pendulum.tomorrow('UTC')

aq_fields = {
    'fmisid': np.int32,
    'time': np.datetime64,
    'AQINDEX_PT1H_avg': np.float64,
    'PM10_PT1H_avg': np.float64,
    'PM25_PT1H_avg': np.float64,
    'O3_PT1H_avg': np.float64,
    'CO_PT1H_avg': np.float64,
    'SO2_PT1H_avg': np.float64,
    'NO2_PT1H_avg': np.float64,
    'TRSC_PT1H_avg': np.float64,
}

url = 'https://opendata.fmi.fi/timeseries'

params = {
    'format': 'json',
    'precision': 'double',
    'groupareas': '0',
    'producer': 'airquality_urban',
    'area': 'Uusimaa',
    'param': ','.join(aq_fields.keys()),
    'starttime': start_time.isoformat(timespec="seconds"),
    'endtime': end_time.isoformat(timespec="seconds"),
    'tz': 'UTC',
}

data = requests.get(url, params=params).json()

In [3]:
df = pd.DataFrame(data).astype(aq_fields)
df = df.set_index(['fmisid', 'time'])
df[0:10]

AttributeError: 'DataFrame' object has no attribute 'set_data_type'

In [ ]:
df.to_parquet('data/airquality.parquet', compression='zstd')